# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [86]:
text = open('peace.txt', 'r', encoding="utf-8").read()[2:]
len(text)

3227579

In [87]:
import string
import re

def preprocess_text(text):
    # TODO
    # make lowercase
    # replace all punctuation except dots with spaces
    # collapse multiple spaces into one '   ' -> ' '
    text = text.lower()
    text = re.sub(r'[^\w\d_.]+', ' ', text)

    
    return text

text = preprocess_text(text)
assert len(text) == 3141169, len(text)

In [88]:
text = text.split('.')
text = [x.strip() for x in text]

In [456]:
from collections import Counter
#import nltk
from sklearn.base import TransformerMixin


class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        # TODO
        # tokenize text by symbols and fill in self.itos and self.stoi
        text = list(" ".join(text)) 
        self.itos = [s for i, s in enumerate(set(text))]
        self.stoi = {s:i for i, s in enumerate(self.itos)}
        #text = # TODO
        
        while len(self.itos) < self.vocab_size:
            # TODO
            # count bigram freqencies in the text
            bigrams = Counter([tuple(text[i:i+2]) for i in range(len(text)-1)])
            new_token = bigrams.most_common(1)[0][0]
            # most common bigram in the text
            new_id = len(self.itos)
            
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            
            # find occurences of the new_token in the text and replace them with new_id
            upd = lambda a, b: new_id if (a, b) == new_token else a
            end = text[-1]
            text = [upd(*text[i:i+len(new_token)]) for i in range(len(text)-1) if tuple(text[max(i-1,0):i+len(new_token)-1]) != new_token]
            if len(text) > 1 and (text[-2], text[-1]) != new_token:
                text.append(end)
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        text = [[self.stoi[s] for s in x] for x in text] # TODO tokenize text by symbols using self.stoi
        for token_id, token in enumerate(self.itos):
            # find occurences of the token in the text and replace them with token_id
            if isinstance(token, str):
                continue
            for i, x in enumerate(text):
                upd = lambda a, b: token_id if (self.itos[a], self.itos[b]) == token else a
                text[i] = [upd(*x[i:i+2]) for i in range(len(x)-1) if not (i > 0 and (self.itos[x[i-1]], self.itos[x[i]]) == token)]      
                if x and len(x) > 1 and (self.itos[x[-2]], self.itos[x[-1]]) != token:
                    text[i].append(x[-1])
                    
        return text
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        if isinstance(tok, int):
            if isinstance(self.itos[tok], str):
                return self.itos[tok]
            else:
                return self.decode(self.itos[tok])
        else:
            return ''.join([x if isinstance(x, str) else self.decode(x) for x in tok])
        
        return result
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
        
        
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [461]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [483]:
import numpy as np
        
    
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = np.zeros(self.vocab_size**3)# TODO create array for storing 3-gram counters
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        cut = a*self.vocab_size**2 + b*self.vocab_size
        p = (self.proba[cut:cut+self.vocab_size] + self.delta)**(1/tau)
        result = p / np.sum(p)
        return result
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        cut = a*self.vocab_size**2 + b*self.vocab_size
        p = (self.proba[cut:cut+self.vocab_size] + self.delta)**(1/tau)
        result = p[c] / np.sum(p) # TODO approximate probability by counters
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        cnt = Counter()
        for i, x in enumerate(text):
            x_trigrams = [(x[i], x[i+1], x[i+2]) for i in range(len(x)-2)]
            cnt_ = Counter(x_trigrams)
            cnt.update(cnt_)
        cnt = dict(cnt)
        trigram_to_id = lambda trigram: trigram[0]*self.vocab_size**2 + trigram[1]*self.vocab_size + trigram[2]
        for trigram, k in cnt.items():
            self.proba[trigram_to_id(trigram)] = k
        # TODO count 3-grams in the text
        
        return self
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [484]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """

    beam = [(input_seq, 0)]
            # TODO store in beam tuples of current sequences and their log probabilities
    
    for i in range(max_len):
        candidates = []
        candidates_proba = []
        for snt, snt_proba in beam:
            if np.argmax(snt[-1]) == end_token:
                continue
            else:    
                proba = lm.infer(*snt[-2:], tau) # probability vector of the next token
                best_k = proba.argsort()[-k:][::-1] # top-k most probable tokens
                # TODO update candidates' sequences and corresponding probabilities
                candidates += [snt + [int(x)] for x in best_k]
                candidates_proba += [snt_proba + x for x in np.log(proba[best_k])]

        beam = [(candidates[i], candidates_proba[i]) for i in np.argsort(candidates_proba).astype(int)[-k:][::-1]]
                # select top-k most probable sequences from candidates
    return beam
    

In [485]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for (snt, snt_proba) in result:
    print(bpe.decode(snt), np.exp(snt_proba))

horse was she said  0.404950894628
horse was said not  0.160088924945
horse was she was s 0.143084119266
horse was she cound 0.0359152963307
horse was she cound  0.0343985108214
horse was she count 0.0325837141851
horse was it was sh 0.0283735835494
horse was she was i 0.0100806863066
horse was not been  0.00964591734276
horse the counder and  0.00863477565772


In [486]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for (snt, snt_proba) in result:
    print(bpe.decode(snt), np.exp(snt_proba))

her and she said  0.306517182991
her and with a sm 0.265585036691
her and she was s 0.108303850541
her and said not  0.0858167465544
her and she cound 0.0271851614693
her and she cound  0.0260370696199
her and she count 0.0246634058991
her he said not  0.0174606918544
her and she was i 0.00763031668853
her and with his f 0.00571154702548


In [487]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
# TODO print decoded generated strings and their probabilities
for (snt, snt_proba) in result:
    print(bpe.decode(snt), np.exp(snt_proba))

what the said not  0.000164128762823
what the prince and  0.000112341770105
what the could not  0.000110316090872
what the prince and 6.91333969874e-05
what the princess  5.46421543915e-05
what the from the s 4.6012404354e-05
what the from the w 4.18534009555e-05
what the said not 4.11379190022e-05
what the from the c 4.03939089977e-05
what the prince of  3.79663413733e-05


In [488]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for (snt, snt_proba) in result:
    print(bpe.decode(snt), np.exp(snt_proba))

gun and she said n 0.310529906882
gun and with a smi 0.269050396288
gun and she was sh 0.0885500030937
gun and said not b 0.0715415976656
gun and she counder 0.0275408028993
gun and she counte 0.024986460397
gun and she cound hi 0.0240626634667
gun and she was sa 0.0208103470787
gun been he said  0.0144819501587
gun and said not s 0.0138707341052


In [491]:
def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    result = np.exp(-np.log(2)*np.mean([np.log(lm.get_proba(*snt[i:i+3])) for i in range(len(snt)-2)]))
                                        #TODO perplexity for the sentence
    return result

perplexity(tokenized_text[0], lm)

5.7347091224849125